In [79]:
import pandas as pd
import spacy
import ast
import re
from pprint import pprint
import json
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from sklearn import linear_model
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, r2_score, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from scipy import sparse
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yubozhang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
reviews = []
with open('scraped_reviews_fb_msngr_ytm.txt', 'r') as f: #, encoding='utf-8'
    reviews = f.read()

reviews = re.sub('\]', '', reviews)
reviews = re.sub('\[', '', reviews)
reviews = re.sub('\n', '', reviews)
reviews = re.sub("'", '"', reviews)
reviews = re.sub('    ', ' ', reviews)
reviews = re.sub(' {', '{', reviews)
reviews = re.sub(' }', '}', reviews)
reviews = re.sub('{ ', '{', reviews)
reviews = re.sub('} ', '}', reviews)
reviews = re.sub('}  {', '}, {', reviews)
reviews = re.sub('}{', '}, {', reviews)
reviews = re.sub('id:', '"id":', reviews)
reviews = re.sub('userName:', '"userName":', reviews)
reviews = re.sub('userImage:', '"userImage":', reviews)
reviews = re.sub('date:', '"date":', reviews)
reviews = re.sub('url:', '"url":', reviews)
reviews = re.sub('score:', '"score":', reviews)
reviews = re.sub('title:', '"title":', reviews)
reviews = re.sub('text:', '"text":', reviews)
reviews = re.sub('replyDate:', '"replyDate":', reviews)
reviews = re.sub('replyText:', '"replyText":', reviews)
reviews = re.sub(': undefined', ': "undefined"', reviews)

In [3]:
review_strings = [r+'}' for r in reviews.split('},')]
review_strings[-1] = review_strings[-1][:-1]

In [4]:
review_list = []
for r in review_strings:
    try:
        review_list.append(json.loads(r))
    except:
        pass

In [83]:
raw_corpus = []
for i in range(1000): # change to larger number
    raw_corpus.append(review_list[i][u'text'])

In [84]:
data = raw_corpus

In [98]:
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
data = [re.sub("\"", "", sent) for sent in data]
data = [re.sub("_", "", sent) for sent in data]

data = [re.sub("good'", "", sent) for sent in data]
data = [re.sub("app", "", sent) for sent in data]
data = [re.sub("nice", "", sent) for sent in data]
pprint(data[0])


u'Its quick and quite impressive '


In [99]:
def sent_to_words(sentences):
    
    yield(gensim.utils.simple_preprocess(str(sentences), deacc=True)) 

In [100]:
data_words = list(sent_to_words(data))

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
print(trigram_mod[bigram_mod[data_words[0][:30]]])

[u'its', u'quick', u'and', u'quite', u'impressive', u'good', u'need', u'it', u'lovely', u'good', u'app', u'very', u'slow', u'galt', u'language', u'bolne', u'valo', u'ko', u'permanently', u'kasa', u'block', u'krta', u'pls', u'answer', u'me', u'peanut', u'butter', u'and', u'jelly', u'good']


In [101]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [102]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#print(data_lemmatized[:1])

In [103]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])

In [104]:
id2word[0]

u'aap'

In [105]:
# Human readable format of corpus (term-frequency)
#[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [106]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


In [107]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  u'0.002*"good" + 0.002*"nice" + 0.002*"love" + 0.002*"great" + 0.002*"update" + 0.002*"not" + 0.002*"messenger" + 0.002*"super" + 0.002*"friend" + 0.002*"video"'),
 (1,
  u'0.003*"good" + 0.002*"nice" + 0.002*"not" + 0.002*"love" + 0.002*"great" + 0.002*"update" + 0.002*"messenger" + 0.002*"call" + 0.002*"friend" + 0.002*"work"'),
 (2,
  u'0.002*"good" + 0.002*"love" + 0.002*"nice" + 0.002*"great" + 0.002*"not" + 0.002*"update" + 0.002*"messenger" + 0.002*"call" + 0.002*"friend" + 0.002*"cc"'),
 (3,
  u'0.002*"good" + 0.002*"nice" + 0.002*"love" + 0.002*"not" + 0.002*"great" + 0.002*"update" + 0.002*"messenger" + 0.002*"work" + 0.002*"call" + 0.002*"cc"'),
 (4,
  u'0.002*"good" + 0.002*"nice" + 0.002*"love" + 0.002*"great" + 0.002*"not" + 0.002*"messenger" + 0.002*"call" + 0.002*"update" + 0.002*"work" + 0.002*"thank"'),
 (5,
  u'0.002*"good" + 0.002*"nice" + 0.002*"love" + 0.002*"not" + 0.002*"great" + 0.002*"call" + 0.002*"update" + 0.002*"friend" + 0.002*"work" + 0.002*"messe

In [108]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
19     97.419861        1       1  0.142987  0.060271
10      2.560507        1       2  0.129528 -0.067505
13      0.001095        1       3  0.003287  0.005608
17      0.001094        1       4 -0.003462  0.003817
7       0.001093        1       5 -0.007288  0.002679
12      0.001093        1       6 -0.009850  0.001938
18      0.001093        1       7 -0.010266  0.001835
16      0.001092        1       8 -0.014587  0.000496
6       0.001091        1       9 -0.015410  0.000313
1       0.001090        1      10 -0.017637 -0.000264
2       0.001090        1      11 -0.018012 -0.000458
5       0.001090        1      12 -0.018344 -0.000490
8       0.001090        1      13 -0.018523 -0.000506
15      0.001089        1      14 -0.019359 -0.000877
11      0.001089        1      15 -0.019513 -0.000864
0       0.001089        1      16 -0.020028 -0.001053
14      0.001088        1      17 -0.020331 -0.001074
4       0.001088        1      18 -0.020406 -0.001150
3       0.001088        1      19 -0.020628 -0.001109
9       0.001085        1      20 -0.022156 -0.001609, topic_info=     Category        Freq       Term       Total  loglift  logprob
term                                                              
195   Default  250.000000       good  250.000000  30.0000  30.0000
664   Default   74.000000       nice   74.000000  29.0000  29.0000
82    Default   54.000000       love   54.000000  28.0000  28.0000
69    Default   41.000000      great   41.000000  27.0000  27.0000
202   Default   40.000000        not   40.000000  26.0000  26.0000
576   Default   34.000000     update   34.000000  25.0000  25.0000
233   Default   29.000000  messenger   29.000000  24.0000  24.0000
102   Default   21.000000       call   21.000000  23.0000  23.0000
119   Default   20.000000       work   20.000000  22.0000  22.0000
317   Default   18.000000       need   18.000000  21.0000  21.0000
182   Default   19.000000      thank   19.000000  20.0000  20.0000
42    Default   17.000000      video   17.000000  19.0000  19.0000
130   Default   17.000000    awesome   17.000000  18.0000  18.0000
579   Default   18.000000      super   18.000000  17.0000  17.0000
290   Default   19.000000     friend   19.000000  16.0000  16.0000
266   Default   17.000000      think   17.000000  15.0000  15.0000
120   Default   17.000000        can   17.000000  14.0000  14.0000
188   Default   15.000000    amazing   15.000000  13.0000  13.0000
477   Default   15.000000       chat   15.000000  12.0000  12.0000
432   Default   14.000000         cc   14.000000  11.0000  11.0000
392   Default   14.000000        see   14.000000  10.0000  10.0000
524   Default   13.000000       easy   13.000000   9.0000   9.0000
383   Default   14.000000        bad   14.000000   8.0000   8.0000
278   Default   13.000000    message   13.000000   7.0000   7.0000
374   Default   12.000000         do   12.000000   6.0000   6.0000
488   Default   11.000000          s   11.000000   5.0000   5.0000
591   Default   11.000000    nothing   11.000000   4.0000   4.0000
567   Default   11.000000         be   11.000000   3.0000   3.0000
376   Default   10.000000        get   10.000000   2.0000   2.0000
24    Default    9.000000   facebook    9.000000   1.0000   1.0000
...       ...         ...        ...         ...      ...      ...
675   Topic20    0.000033     longer    0.460034   1.8905  -6.5265
664   Topic20    0.000035       nice   74.301201  -3.1470  -6.4795
82    Topic20    0.000034       love   54.989361  -2.8567  -6.4902
202   Topic20    0.000034        not   40.600834  -2.5563  -6.4931
69    Topic20    0.000034      great   41.719791  -2.5841  -6.4937
576   Topic20    0.000034     update   34.592983  -2.4041  -6.5010
233   Topic20    0.000034  messenger   29.529943  -2.2466  -6.5018
102   Topic20    0.000034       call   21.755514  -1.9470  -6.5078
42    Topic20    0.000